In [ ]:
import pandas as pd
import plotly.express as px
import numpy as np

# Diferencias de exportaciones por omega y criterio de simulacion

In [ ]:
## CARGA DE DATOS PARA MEDIR DIFERENCIAS ENRTRE CRITERIOS
exports_diff_dfs = []
exports_global_dfs = []
exports_local_dfs = []
for omega in ("0.1", "0.2", "0.3", "0.4", "0.5", "0.6", "0.8", "0.9", "0.55", "0.65"):
    exports_global_df = pd.read_csv(f"experimentacion/metrics-true-{omega}/exports.csv")
    exports_global_df["omega"] = float(omega)
    exports_global_dfs.append(exports_global_df)

    exports_local_df = pd.read_csv(f"experimentacion/metrics-false-{omega}/exports.csv")
    exports_local_df["omega"] = float(omega)
    exports_local_dfs.append(exports_local_df)

    exports_diff_df = exports_global_df.set_index(["country", "generation"]) - exports_local_df.set_index(["country", "generation"])
    exports_diff_df["omega"] = float(omega)

    exports_diff_dfs.append(exports_diff_df)

exports_diff_df = pd.concat(exports_diff_dfs)
exports_diff_df.sort_values("omega", inplace=True)

exports_global_df = pd.concat(exports_global_dfs)
exports_global_df.sort_values("omega", inplace=True)

exports_local_df = pd.concat(exports_local_dfs)
exports_local_df.sort_values("omega", inplace=True)

Veamos primero cual es la tendencia de diferencias en exportaciones actualizando $\Phi$ en cada generacion (llamado criterio de simulacion global) versus no realizar esta actualizacion (llamado criterio de simulacion local).

En la bibliografia se menciona que esta actualizacion no trae beneficios claros. Tener en cuenta que actualizar $\Phi$ corresponde a volver a categorizar los productos.

Para esto elegimos visualizar la el avance de la diferencia absoluta segun cada umbral $\Omega$ para distintos paises. A medida avanza la simulacion, dado que es un proceso de difusion, se aproximan asintotas sobre las metricas observadas.

In [ ]:
paises = ["ARG"]
# TODO: fijar los paises
paises = set(paises + [country for country, _ in np.random.choice(exports_diff_df.index, 5 - len(paises))])
for pais in sorted(paises):
    df = exports_diff_df.loc[pais].reset_index().sort_values(["generation","omega"])
    fig = px.line(df,
                x="generation",
                y="count_exports",
                color="omega",
                title=f"Diferencia en cantidad de exportaciones para {pais} y omega segun criterio de simulacion",
                labels={"count_exports": "Diferencia de exportaciones", "generation": "Generacion", "country": "Pais"})
    fig.show()

Se observa que, para todos los paises, la diferencia se estabiliza en las ultimas generaciones (10ma generacion en adelante), por lo que tomaremos las comparativas con la menor divergencia (la ultima generacion disponible).

Veamos con mayor detalle como es la distribucion segun $\Omega$ (umbral de "descubrimiento").

In [ ]:
max_diff_df = (
    exports_diff_df
    .reset_index()
    .set_index(["generation", "country", "omega"])
    .loc[max(generation for _, generation in exports_diff_df.index)]
    .max(axis=1)
)
max_diff_df.rename("difference", inplace=True)
fig = px.box(
    max_diff_df.reset_index(),
    x="omega",
    y="difference",
    title=f"Distribucion de diferencias segun criterio de simulacion en cantidad de exportaciones x omega (ultima generacion)",
    labels={"difference": "Diferencia de exportaciones", "omega": "Omega"})
fig.show()

Claramente los mas interesantes de analizar son los valores de $\Omega = 0.5, 0.55, 0.6 $ y $ 0.65$. Esto se condice con la idea de que a valores muy pequenos de $\Omega$ el descubrimiento es facil y a valores muy elevados es muy dificil, por lo que las alternativas globales y locales pecan en los mismos aspectos ($\Omega$ seria mas significativo que actualizar $\Phi$).


#### Local vs global para $\Omega$ poco interesante

In [ ]:
# diferencia total en la cantidad de exportaciones de los omegas
UNINTRESTING_OMEGAS = (0.1, 0.2, 0.3, 0.4, 0.6, 0.8, 0.9)
df_ = exports_diff_df[exports_diff_df.omega.isin(UNINTRESTING_OMEGAS)].reset_index().sort_values(["omega", "generation"])
fig = px.line(df_,
        color="country",
        x="generation",
        y="count_exports",
        facet_col="omega", facet_col_wrap=2,
        height=1000,
        width=2000,
        title=f"Evolucion de diferencias segun criterio de simulacion en cantidad de exportaciones (por omega)",
        labels={"count_exports": "Diferencia de exportaciones", "omega": "Omega", "generation": "Generation"})
fig.for_each_annotation(lambda a: a.update(text=a.text.replace("=", " = ")))
fig.update_yaxes(matches=None, range=[0, df_.count_exports.max()])
fig.update_xaxes(matches=None)
fig.show()

Como vemos, no hay cambios de interes fuera de las primeras generaciones para ninguno de los omegas elegidos, por lo que para $\Omega = 0.1, 0.2, 0.3, 0.4, 0.6, 0.8, 0.9$ concluimos que las versiones locales y globales son indistinguibles.

#### Local vs global para $\Omega$ interesante

In [ ]:
INTRESTING_OMEGAS = (0.5, 0.55, 0.6, 0.65)
df_ = exports_diff_df[exports_diff_df.omega.isin(INTRESTING_OMEGAS)].reset_index().sort_values(["omega", "generation"])
fig = px.line(df_,
        color="country",
        x="generation",
        y="count_exports",
        facet_col="omega", facet_col_wrap=2,
        height=1000,
        width=2000,
        title=f"Evolucion de diferencias segun criterio de simulacion en cantidad de exportaciones (por omega)",
        labels={"count_exports": "Diferencia de exportaciones", "omega": "Omega", "generation": "Generation"})
fig.for_each_annotation(lambda a: a.update(text=a.text.replace("=", " = ")))
fig.update_yaxes(matches=None, range=[0, df_.count_exports.max()])
fig.update_xaxes(matches=None)
fig.show()

Cabe notar que para $\Omega = 0.6$ y $0.65$ la el comportamiento entre la version local y la global no sufre grandes cambios, por lo que reclasificamos estos valores como de no interes y subsecuentemente como ambas versiones son indistinguibles para estos valores.

No obstante, $\Omega = 0.5$ y $0.55$ si muestran comportamientos significativamente distintos, por lo que no podemos concluir que la version local y la global tengan indistinguibles. Como lo que esta siendo graficado es global - local y la diferencia es positiva y mayor a 20 productos en la mayoria de los casos, interprestamos que la version global facilita la difusion.

Explicamos el comportamiento indistinguible de los casos clasificados como $\Omega$ poco interesante ya que para estos valores de $\Omega$ la difusion suele alcanzar maximos por ser muy facil difundir o directamente es muy dificil, entonces la retroalimentacion no aporta el suficiente peso como para contrarrestar los efectos negativos en el desarrollo de nuevos productos.

Por ultimo veamos la cantidad de productos exportados. 

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

paises

fig = make_subplots(len(paises), shared_xaxes=True, shared_yaxes=True)

for i, pais in enumerate(paises):
    global_df_ = exports_global_df[exports_global_df.country == pais]
    global_df_.sort_values(["omega", "generation"])
    omegas = set(global_df_.omega)
    for omega in omegas:
        df_ = global_df_[global_df_.omega==omega].sort_values("generation")
        fig.add_trace(
            go.Scatter(
                x=df_.generation,
                y=df_.count_exports,
                mode="lines",
            ),
            row=i+1, col=1,
        )

fig.show()

In [ ]:
exports_global_df

# Pruebas

In [ ]:
exports_global_df_copy = exports_global_df.copy()
exports_global_df_copy["local"] = False
exports_local_df_copy = exports_local_df.copy()
exports_local_df_copy["local"] = True
exports_df = exports_global_df_copy.append(exports_local_df)

In [ ]:
top_20 = exports_df[exports_df.generation == exports_df.generation.max()].nlargest(n=20, columns=["count_exports"]).country
exports_top_20_df = exports_df[exports_df.country.isin(top_20) | (exports_df.country == "ARG")]
fig = px.line(exports_top_20_df,
    x="generation",
    y="count_exports",
    color="country",
    title=f'Productos exportados por pais para los 20 exportadores mas diversificados + ARG (RCA > 1)',
    labels={"count_exports": "Exportaciones", "generation": "Generacion", "couintry": "Pais"},
)
fig.update_yaxes(range=[0, exports_df.count_exports.max()])
fig.show()